In [31]:
qasm_str = """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[11];
u3(1.5509766252323924*pi,0.0*pi,4.0*pi) q[1];
"""

In [32]:
import os
import pennylane as qml
from pennylane.tape import make_qscript, QuantumScript
circuit = qml.from_qasm(qasm_str)

print(qml.draw(circuit)())

1: ──U3(4.87,0.00,12.57)─┤  


In [33]:
from pennylane.transforms import (
    commute_controlled,
    single_qubit_fusion,
    combine_global_phases,
)

new_circuit = commute_controlled(circuit)
print(qml.draw(new_circuit)())

new_circuit = single_qubit_fusion(new_circuit)
print(qml.draw(new_circuit)())

new_circuit = combine_global_phases(new_circuit)
print(qml.draw(new_circuit)())

1: ──U3(4.87,0.00,12.57)─┤  
1: ──U3(4.87,0.00,12.57)─┤  
1: ──U3(4.87,0.00,12.57)─┤  


In [34]:
circuit

<function pennylane_qiskit.converter.load.<locals>._function(*args, params: dict = None, wires: list = None, **kwargs)>

In [43]:
from pennylane.transforms import decompose
dec_circuit = decompose(
    new_circuit, {qml.RX, qml.RY, qml.RZ, qml.CNOT, qml.CZ})

qs = make_qscript(dec_circuit)()

highest_wire = 0
for op in qs:
    print(op.name)
    highest_wire_this_op = max(op.wires) if op.wires else 0
    highest_wire = max(highest_wire, highest_wire_this_op)
print(highest_wire)

new_ops = []
for i in range(highest_wire+1):
    new_ops.append(qml.Identity(wires=[i]))
for op in qs:
    new_ops.append(op)

new_qs = QuantumScript(new_ops)
qasm_str_pennylane = new_qs.to_openqasm(measure_all=False)

print(qasm_str_pennylane)

RZ
RY
RZ
RZ
RZ
1
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
id q[0];
id q[1];
rz(12.566370614359172) q[1];
ry(4.872536771719574) q[1];
rz(-12.566370614359172) q[1];
rz(12.566370614359172) q[1];
rz(0.0) q[1];



In [28]:
from qiskit.quantum_info import Operator
from qiskit.qasm2 import loads

qc_start = loads(qasm_str)
qc_end = loads(qasm_str_pennylane)


op_start = Operator(qc_start)
op_end = Operator(qc_end)

print(op_start.equiv(op_end))

# get final probability distribution

QASM2ParseError: "<input>:6,42: index 1 is out-of-range for register 'q' of size 1"

In [24]:
# store both as files
with open("circuit_start.qasm", "w") as f:
    f.write(qasm_str)

with open("circuit_end.qasm", "w") as f:
    f.write(qasm_str_pennylane)

from mqt import qcec

result = qcec.verify(
    "circuit_start.qasm",
    "circuit_end.qasm",
    transform_dynamic_circuit=True)
equiv = str(result.equivalence)
print(equiv)

not_equivalent


[QCEC] Warning: at least one of the circuits has garbage qubits, but partial equivalence checking is turned off. In order to take into account the garbage qubits, enable partial equivalence checking. Consult the documentation for moreinformation.
